In [1]:
cd /home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files/

/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files


In [2]:
# cd /mnt/storage/data/emaxx/txt_files/

In [36]:
# system/os/regex and basic math functions
import sys
import os
import re
import math
from itertools import combinations as npn
 
# IPython display convenience stuff
from IPython.display import display_html, display_javascript
from IPython import __version__ as ipythonversion
print("IPython: {}".format(ipythonversion))
    
# Set logging level
import logging
logging.getLogger("").setLevel(logging.DEBUG)
 
# numpy for matrix algebra
import numpy as np
print("Numpy: {}".format(np.version.full_version))
 
# scipy for probability distributions and some statistical tests
import scipy as sp
import scipy.stats as stats
print("Scipy: {}".format(sp.version.full_version))
 
# pandas for data manipulation
import pandas as pd
print("Pandas: {}".format(pd.version.version))
 
# Set pandas display options for pretty pretty printing
pd.set_option('html', True, 'precision', 8)
pd.set_option('max_rows',200,'max_columns',50)
 
# pandas web interface for things like FF factors
import pandas.io.data as web
 
# pytables for hdf5 library (on-disk storage)
# import tables as tb
 
# SQLAlchemy for relational db management
import sqlalchemy as sa
print("SQLalchemy: {}".format(sa.__version__))

# matplotlib for plotting and pyplot for MATLAB-style API
import matplotlib as mpl
import matplotlib.pyplot as plt
print("MatPlotLib: {}".format(mpl.__version__))
 
# display plots inline
%matplotlib inline
 
# statsmodels for models with a formula framework similar to R 
import statsmodels.api as sm
print("Statsmodels: {}".format(sm.version.full_version))
 
# datetime for date functions/time series
import datetime as dt
import dateutil

from collections import defaultdict as ddict

debug = False

IPython: 3.0.0
Numpy: 1.9.2
Scipy: 0.15.1
Pandas: 0.15.2
SQLalchemy: 0.9.9
MatPlotLib: 1.4.3
Statsmodels: 0.6.1


## Get file names and count across year/qtr

In [3]:
zulu = ddict(int)
for r,d,f in os.walk('.'):
    if not f: continue
    for fname in f: zulu[fname] += 1

## Check filesizes to find which are same across years

In [4]:
data = pd.DataFrame(index=zulu.keys())

for r,d,f in os.walk('.'):
    if d: continue
    #print(r,d,f)
    yr,qtr = re.search(r"(20\d\d)_Q(\d)", r).groups()
    
    data['{}_{}'.format(yr,qtr)
         ] = pd.Series({fn:os.stat(os.path.join(r,fn)).st_size for fn in f})

samesame = [poo
            for poo in data.index
            if data.transpose()[poo].max()==data.transpose()[poo].min()]
print(samesame)

['BCRSECTS.TXT', 'NONCALL.TXT', 'ENTITY.TXT', 'ASSETCLM.TXT', 'BONDCAT.TXT', 'FIRMTYPE.TXT', 'SECURED.TXT', 'CORPCRED.TXT', 'CPNSTRU.TXT', 'CORPCALL.TXT', 'STATE.TXT', 'NCREDTYP.TXT', 'REDEMETH.TXT']


### Test SAME SAME

This below runs to verify that all the files that have the same content, not just the same filesize. Run once to check, then ignore FOREVER!

In [5]:
txts = {}
for r,d,f in os.walk('.'):
    if not f: continue
    for fname in f:
        if fname not in samesame: continue
        if fname not in txts:
            with open(os.path.join(r,fname)) as fh:
                txts[fname] = fh.read()
        with open(os.path.join(r,fname)) as fh:
            if txts[fname] == fh.read(): continue
        print("NOT SAME SAME!", fname)

# Make function that walks directories

In [3]:
def walk_files(root=None):
    """Returns tuple (root,fname,yr,qtr)"""
    if not root: root="."
    vals = []
    for r,d,f in os.walk(root):
        if d: continue
        try:
            yr,qtr = re.search(r"(20\d\d)_Q(\d)", r).groups()
        except AttributeError as e:
            print(r)
            continue
            #raise e
        for fname in f:
            vals.append((r,fname,yr,qtr))
    return sorted(vals,key=lambda x: "{}_{}".format(x[0],x[1]))

In [4]:
# Remove stupid \x1a EOF marker.
last_yr = ''
for x in walk_files():
    r,fname,yr,qtr = x
    if yr != last_yr:
        print(yr)
        last_yr = yr
    full_path = os.path.join(r,fname)
    with open(full_path, encoding = "ISO-8859-1") as fh:
        tmp = fh.read()
    stripped_tmp = tmp.strip('\r\n \x1a')  # remove b'\x1a'
    if '\x1a' not in stripped_tmp:
        continue
    with open(full_path, 'w', encoding='utf-8') as fh:
        fh.write(stripped_tmp.strip())

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013


## Clean up bad quotes in original files

In [152]:
bad_quote_re = r'(?<![,\n])"(?![,\n])'
for x in walk_files():
    r,fname,yr,qtr = x
    if int(yr) > 2001: break
    if fname not in column_headings.keys(): continue
    with open(os.path.join(r,fname), encoding="ISO-8859-1") as fh:
        grr = fh.read()
        rslts = re.search(bad_quote_re,grr[1:])
        if rslts: # find any " not preceded by a comma
            newgrr = re.sub(bad_quote_re, "'", grr)
            print(fname,yr,qtr,rslts.start(),rslts.end())
            display(HTML("{0}{1}<br>{0}{2}".format('&nbsp;'*10,
                                                   grr[rslts.start()-8:rslts.end()+8],
                                                   newgrr[rslts.start()-8:rslts.end()+8])))

FUND.TXT 2000 3 727928 727929


In [4]:
def strip_dot_zero(csv_string):
    """Pandas prints out 1.0 for all floats, and 
    telling it what are ints is annoying."""

    return re.sub(r'[.]0(?=[,\n])', "", csv_string).strip()

### Column Headings

ISSUERS.TXT: cusip6, name, name_sort, credit_sector, country, nothing, entity, state, ticker

SECMAST.TXT: cusip6, cusip78, issue_desc, coupon_rate, coupon_structure, currency_code, maturity_year, maturity_date, market_sector, collateral_code, cusip_source, private, issue_amt_outstanding, issue_amt_change, num_held, num_buying, num_selling, total_par_held, pledge_code, issue_date, moody, s_and_p, fitch, duff_and_phelps

FUND.TXT: sub_account_id, sub_account_class, sub_account_name, sub_account_name_sort, market_sector, managing_firm_id, sub_account_country, total_par_amt_held, total_number_bonds_held, update_date, trash

HOLDING.TXT: cusip6, cusip78, sub_account_id, par_amount, report_date, par_amount_change, managing_firm_id, acquisition_book_value

FIRM.TXT: managing_firm_id, managing_firm_name, managing_firm_name_sort, managing_firm_phone, managing_firm_fax, managing_firm_loc_addr1, managing_firm_loc_addr2, managing_firm_loc_city, managing_firm_loc_state, managing_firm_loc_zip, managing_firm_mailing_addr1, managing_firm_mailing_addr2, managing_firm_mailing_city, managing_firm_mailing_state, managing_firm_mailing_zip, managing_firm_type, managing_firm_country_code, managing_firm_phone_country_code, managing_firm_total_par_held, managing_firm_number_issues_held

In [17]:
col_str = """
ISSUERS.TXT: cusip6, name, name_sort, credit_sector, country, nothing, entity, state, ticker
SECMAST.TXT: cusip6, cusip78, issue_desc, coupon_rate, coupon_structure, currency_code, maturity_year, maturity_date, market_sector, collateral_code, cusip_source, private, issue_amt_outstanding, issue_amt_change, num_held, num_buying, num_selling, total_par_held, pledge_code, issue_date, moody, s_and_p, fitch, duff_and_phelps
FUND.TXT: sub_account_id, sub_account_class, sub_account_name, sub_account_name_sort, market_sector, managing_firm_id, sub_account_country, total_par_amt_held, total_number_bonds_held, update_date, _
HOLDING.TXT: cusip6, cusip78, sub_account_id, par_amount, report_date, par_amount_change, managing_firm_id, acquisition_book_value
FIRM.TXT: managing_firm_id, managing_firm_name, managing_firm_name_sort, managing_firm_phone, managing_firm_fax, managing_firm_loc_addr1, managing_firm_loc_addr2, managing_firm_loc_city, managing_firm_loc_state, managing_firm_loc_zip, managing_firm_mailing_addr1, managing_firm_mailing_addr2, managing_firm_mailing_city, managing_firm_mailing_state, managing_firm_mailing_zip, managing_firm_type, managing_firm_country_code, managing_firm_phone_country_code, managing_firm_total_par_held, managing_firm_number_issues_held
""".strip().split('\n')

column_headings = {ln.split(': ')[0]:ln.split(': ')[1].split(', ') for ln in col_str}

In [5]:
# . denotes numeric
keep_col_str = """
ISSUERS.TXT: cusip6, year., quarter., name, name_sort, country, ticker
SECMAST.TXT: cusip6, cusip78, year., quarter., issue_desc, coupon_rate, coupon_structure, currency_code, maturity_year., maturity_date, market_sector, collateral_code, cusip_source, private, issue_amt_outstanding., issue_amt_change., num_held., num_buying., num_selling., total_par_held., pledge_code, issue_date, moody, s_and_p, fitch, duff_and_phelps
FUND.TXT: sub_account_id., year., quarter., sub_account_class, sub_account_name, sub_account_name_sort, managing_firm_id., sub_account_country, total_par_amt_held., total_number_bonds_held., update_date
HOLDING.TXT: cusip6, cusip78, year., quarter., sub_account_id., par_amount., report_date, par_amount_change., managing_firm_id., acquisition_book_value.
FIRM.TXT: managing_firm_id., year., quarter., managing_firm_name, managing_firm_name_sort, managing_firm_type, managing_firm_country_code, managing_firm_total_par_held., managing_firm_number_issues_held.
""".strip().split('\n')

keep_cols = {ln.split(': ')[0]:ln.split(': ')[1].split(', ') for ln in keep_col_str}

## Do all the files that are simple.
Ignores SECMAST, HOLDING, and FUND

In [81]:
!head -n 1 FUND.TXT

sub_account_id,year,quarter,sub_account_class,sub_account_name,sub_account_name_sort,managing_firm_id,sub_account_country,total_par_amt_held,total_number_bonds_held,update_date


In [86]:
if False:
    raise
    
for r,fname,yr,qtr in walk_files():
    first = yr+qtr=='20001'
    if fname not in column_headings.keys():
        continue
    #if fname in ('SECMAST.TXT', 'HOLDING.TXT'): continue
    if yr != oldyr:
        print(r,fname,yr,qtr)
        oldyr = yr
    new_data = pd.read_csv(os.path.join(r,fname), names=column_headings[fname]
                           ,dtype=None if fname != 'SECMAST.TXT' else {'duff_and_phelps':pd.np.object}
                           ,header=None, index_col=False, low_memory=False)
    
    if fname == 'HOLDING.TXT':
        new_data[column_headings[fname][2:]] = new_data[column_headings[fname][2:]
                                                       ].convert_objects(convert_numeric=True)
    if fname in ('FIRM.TXT', 'HOLDING.TXT', 'FUND.TXT'):
        new_data['managing_firm_id'] = new_data.managing_firm_id.apply(lambda x: -99 if x == 'CO-MANAGED' else x)
    
    new_data['year'], new_data['quarter'] = yr,qtr
    
    new_txt = new_data[[x.strip('.') for x in keep_cols[fname]]
                      ].to_csv(header=first, index=False, encoding="utf-8")
    if first: print(fname,new_txt.split('\n')[0])
    with open(os.path.join(fname), 'a' if not first else 'w', encoding="utf-8") as fh:
        if not first: fh.write("\n")
        fh.write(strip_dot_zero(new_txt).strip())

./2000_Q1/Q1 FIRM.TXT 2000 1
FIRM.TXT managing_firm_id,year,quarter,managing_firm_name,managing_firm_name_sort,managing_firm_type,managing_firm_country_code,managing_firm_total_par_held,managing_firm_number_issues_held
FUND.TXT sub_account_id,year,quarter,sub_account_class,sub_account_name,sub_account_name_sort,managing_firm_id,sub_account_country,total_par_amt_held,total_number_bonds_held,update_date
HOLDING.TXT cusip6,cusip78,year,quarter,sub_account_id,par_amount,report_date,par_amount_change,managing_firm_id,acquisition_book_value
ISSUERS.TXT cusip6,year,quarter,name,name_sort,country,ticker
SECMAST.TXT cusip6,cusip78,year,quarter,issue_desc,coupon_rate,coupon_structure,currency_code,maturity_year,maturity_date,market_sector,collateral_code,cusip_source,private,issue_amt_outstanding,issue_amt_change,num_held,num_buying,num_selling,total_par_held,pledge_code,issue_date,moody,s_and_p,fitch,duff_and_phelps
./2001_Q1/Q1 FIRM.TXT 2001 1
./2002_Q1/Q1 FIRM.TXT 2002 1
./2003_Q1/Q1 FIRM.TXT

# Add to postgres server

In [34]:
def type_formatter(name):
    if '.' in name:
        return "{} integer DEFAULT 0".format(name.split('.')[0])
    return "{} text".format(name)
    
def make_psql_create_string(fname, col_dict):
    name = fname.split('.')[0].lower()
    return """
    CREATE TABLE "{}" ({})
    WITH ( OIDS=FALSE );
    """.format(name
               ,",\n\t".join([type_formatter(col) for col in col_dict[fname]]))

In [87]:
ROOTDIR = '/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files/'
if False:
    engine = sa.create_engine('postgresql+psycopg2:///kbd?host=/var/run/postgresql')
    for fname,cols in column_headings.items():
        tmp_name = fname.split('.')[0].lower()
        with engine.connect() as connection:
            connection.execute("DROP TABLE IF EXISTS {};".format(tmp_name))
            query = make_psql_create_string(fname, keep_cols)
            connection.execute(query)
        #with engine_create.connect() as connection:
            query = "COPY {0} FROM '{1}' WITH CSV HEADER;".format(
                        tmp_name, os.path.join(ROOTDIR, fname))
            print('\\'+query)
            #results = connection.execute(query)

\COPY fund FROM '/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files/FUND.TXT' WITH CSV HEADER;
\COPY issuers FROM '/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files/ISSUERS.TXT' WITH CSV HEADER;
\COPY holding FROM '/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files/HOLDING.TXT' WITH CSV HEADER;
\COPY firm FROM '/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files/FIRM.TXT' WITH CSV HEADER;
\COPY secmast FROM '/home/gaulinmp/Dropbox/Documents/School/Projects/KBP_katrina_bond_to_private/data/emaxx/txt_files/SECMAST.TXT' WITH CSV HEADER;
